In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import gensim

In [2]:
df=pd.read_csv('train.csv')
df=df.sample(frac=1).reset_index(drop=True) # Shuffling the dataset
df.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id49002,"I know this is a boutique hotel, but there are...",Google Chrome,Desktop,not happy
1,id11557,I agree with Rednecktreveller that this would ...,Google Chrome,Desktop,not happy
2,id18016,When we arrived at the entrance we realized th...,Firefox,Tablet,happy
3,id24085,Sheraton has never been my favourite hotel cha...,Edge,Tablet,not happy
4,id40980,Have stayed numerous times at other locations ...,Mozilla,Tablet,not happy


In [3]:
df=df.iloc[:5000,:] # selecting only 5000 rows as large file will not run in the system
df.shape

(5000, 5)

In [4]:
df['Is_Response'].value_counts()

happy        3403
not happy    1597
Name: Is_Response, dtype: int64

In [5]:
# Dropping columns like user_id, browser used and device used which is of no much importance
df.drop(['User_ID', 'Browser_Used', 'Device_Used'],axis=1,inplace=True) 
df.head()

,Description,Is_Response
0,"I know this is a boutique hotel, but there are...",not happy
1,I agree with Rednecktreveller that this would ...,not happy
2,When we arrived at the entrance we realized th...,happy
3,Sheraton has never been my favourite hotel cha...,not happy
4,Have stayed numerous times at other locations ...,not happy


In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from string import punctuation
import string
import re
wordnet=WordNetLemmatizer()

In [7]:
corpus=[]
for i in range(0,len(df)):
    review=re.sub('[^a-zA-Z]',' ', df['Description'][i])
    review=review.lower()
    review=review.split()
    review=[wordnet.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

#### Model building


In [8]:

df['Is_Response']=pd.get_dummies(df['Is_Response'])


In [9]:
df['Is_Response'].value_counts()

1    3403
0    1597
Name: Is_Response, dtype: int64

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

vec=TfidfVectorizer()
bayes=GaussianNB()
x=vec.fit_transform(corpus).toarray()



In [19]:
x.shape

(5000, 15862)

In [11]:
y=pd.DataFrame(df['Is_Response'])

In [12]:
# train and predict
train_x,test_x,train_y,test_y=train_test_split(x,y,random_state=42,test_size=0.20)



In [13]:
bayes.fit(train_x,train_y)
pred=bayes.predict(test_x)
acc_score=accuracy_score(pred,test_y)
print("Accuracy Score is", acc_score)
print(confusion_matrix(pred,test_y))
print(classification_report(pred,test_y))

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy Score is 0.573
[[195 295]
 [132 378]]
              precision    recall  f1-score   support

           0       0.60      0.40      0.48       490
           1       0.56      0.74      0.64       510

    accuracy                           0.57      1000
   macro avg       0.58      0.57      0.56      1000
weighted avg       0.58      0.57      0.56      1000



In [14]:
# Testing results on the test data
testdf=pd.read_csv("test.csv")
testdf.head()

,User_ID,Description,Browser_Used,Device_Used
0,id80132,Looking for a motel in close proximity to TV t...,Firefox,Mobile
1,id80133,Walking distance to Madison Square Garden and ...,InternetExplorer,Desktop
2,id80134,Visited Seattle on business. Spent - nights in...,IE,Tablet
3,id80135,This hotel location is excellent and the rooms...,Edge,Mobile
4,id80136,This hotel is awesome I love the service Antho...,Mozilla,Mobile


In [15]:
testdf.drop(['User_ID', 'Browser_Used', 'Device_Used'],axis=1,inplace=True) 
testdf=testdf.sample(frac=1).reset_index(drop=True) # Shuffling the dataset
testdf=testdf.iloc[:5000,:] # selecting only 5000 rows as large file will not run in the system
testdf.shape

(5000, 1)

In [16]:
testcorpus=[]
for i in range(0,len(testdf)):
    review=re.sub('[^a-zA-Z]',' ', testdf['Description'][i])
    review=review.lower()
    review=review.split()
    review=[wordnet.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    testcorpus.append(review)

In [21]:
testdata=vec.fit_transform(testcorpus).toarray()

In [22]:
testdata.shape

(5000, 15781)

In [18]:
predtest=bayes.predict(testdata)
predtest.to_csv('prediction_testdata.csv')

ValueError: operands could not be broadcast together with shapes (5000,15781) (15862,) 